## SSL Enabled Requests

Install seldon with ingress provider.


In [ ]:
%%bash
kubectl create ns cert-manager
helm install \
  cert-manager jetstack/cert-manager \
  --namespace cert-manager \
  --version v0.15.1 \
  --set installCRDs=true

#### Deploying self signed certificate

In [8]:
%%bash
kubectl apply -f - << END
apiVersion: cert-manager.io/v1alpha2
kind: Issuer
metadata:
  name: selfsigned-issuer
  namespace: default
spec:
  selfSigned: {}
END

issuer.cert-manager.io/selfsigned-issuer created


In [4]:
!kubectl get issuer 

NAME                READY   AGE
selfsigned-issuer   True    23s


In [9]:
%%bash
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: sklearn
spec:
  name: iris
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
    name: default
    replicas: 1
    ssl:
      certSpecOverrides:
        issuerRef:
          name: selfsigned-issuer
        dnsNames:
        - example.com

certificate.cert-manager.io/selfsigned-crt created


In [14]:
!kubectl get certificate

NAME                   READY   SECRET                 AGE
sklearn-default-cert   True    sklearn-default-cert   3m1s


In [ ]:
%%bash
curl -v -X POST -H 'Content-Type: application/json' \
    -d '{"data": {"ndarray": [[1,2,3,4]]}}' \
    https://localhost:80/seldon/default/sklearn/api/v1.0/predictions

In [ ]:
%%bash
cd ../../../executor/proto && \
grpcurl \
         -rpc-header seldon:sklearn -rpc-header namespace:default \
        -d '{"data": {"ndarray": []}}' \
         -proto prediction.proto  127.0.0.1:443 seldon.protos.Seldon/Predict